```shell

```

# Zotero PDF生成大模型训练数据集

## 安装依赖

### 安装 PyMuPDF
```shell
pip install PyMuPDF
```

### 安装Pyzotero
```shell
pip install pyzotero
# 6zWcYYQ2eiVYDUokDh1RsI0R
```

### 安装 langchain & openai
```shell
pip install openai langchain langchain-community tiktoken faiss-cpu
```

## 初始化KEY

In [6]:
import os

os.environ["OPENAI_API_KEY"] = ''
os.environ["OPENAI_BASE_URL"] = 'https://aihubmix.com/v1'

use_openai = True
QUESTION_GENERATOR_MODEL = "aihubmix-Llama-3-3-70B-Instruct"
ANSWER_GENERATOR_MODEL = "aihubmix-Llama-3-3-70B-Instruct"
THINKING_MODEL = "aihubmix-Llama-3-3-70B-Instruct"

In [7]:
import os
# Remove OPENAI_API_KEY from environment variables
if "OPENAI_API_KEY" in os.environ:
    del os.environ["OPENAI_API_KEY"]

# Set Ollama base URL for local usage
os.environ["OPENAI_BASE_URL"] = "http://localhost:11434"

use_openai = False
EMBEDDING_MODEL = "deepseek-r1:70b"
QUESTION_GENERATOR_MODEL = "deepseek-r1:70b"
ANSWER_GENERATOR_MODEL = "deepseek-r1:70b"
THINKING_MODEL = "deepseek-r1:70b"


In [8]:
DEBUG = True

## 读取zotero的信息

In [ ]:
from pyzotero import zotero

zot = zotero.Zotero(
    library_id='15307681',
    library_type='user',
    local=True
)

# 获取所有collections
collections = zot.collections()

# 查找"同步辐射光源"分类
target_collection = None
for collection in collections:
    if collection['data']['name'] == '同步辐射光源':
        target_collection = collection
        break

if  not target_collection:
    print("未找到'同步辐射光源'分类")

    # 获取该分类下的所有条目
items = zot.collection_items(target_collection['key'])

    # 打印分类信息和条目信息
print(f"Collection: {target_collection['data']['name']}")
print(f"Collection Key: {target_collection['key']}")
print(f"Collection Items: {len(items)}")
print("\nItems in collection:")
for item in items:
    print(f"\nTitle: {item['data']['title']}")
    print(f"Type: {item['data']['itemType']}")
    if 'abstractNote' in item['data']:
        print(f"Abstract: {item['data']['abstractNote']}")


## 获取所有附件信息

In [ ]:
# 获取所有条目的附件信息
import urllib


attachments = []
print(f"Total items: {len(items)}")

for item in items:
    # 获取每个条目的附件
    item_attachments = zot.children(item['key'])
    for attachment in item_attachments:
        if attachment['data']['itemType'] == 'attachment' and 'contentType' in attachment['data']:
            # 只处理PDF附件
            if attachment['data']['contentType'] == 'application/pdf':
                # 从enclosure链接中获取本地文件路径
                file_path = urllib.parse.unquote(attachment['links']['enclosure']['href'].replace('file:///', '')) if 'enclosure' in attachment['links'] else None

                attachment_info = {
                    'parent_title': item['data']['title'],  # 父条目标题
                    'parent_key': item['key'],              # 父条目key
                    'attachment_key': attachment['key'],    # 附件key
                    'filename': attachment['data']['filename'],  # 文件名
                    'file_path': file_path,                # 本地文件路径
                    'content_type': attachment['data']['contentType'],  # 内容类型
                    'added_date': attachment['data']['dateAdded'],     # 添加日期
                    'modified_date': attachment['data']['dateModified'],# 修改日期
                    'md5': attachment['data'].get('md5', ''),         # MD5校验和
                }
                attachments.append(attachment_info)

print(f"\nFound {len(attachments)} PDF attachments:")
for attachment in attachments:
    print(f"\nParent Item: {attachment['parent_title']}")
    print(f"Filename: {attachment['filename']}")
    print(f"File Path: {attachment['file_path']}")
    print(f"Added Date: {attachment['added_date']}")
    print(f"MD5: {attachment['md5']}")


## 解析PDF信息

In [ ]:
# 解析PDF信息
try:
    import fitz # PyMuPDF
    import os

    print("\nParsing PDF contents...")

    # 检查PDF文件是否存在并可访问
    valid_attachments = []
    for attachment in attachments:
        if attachment['file_path'] and os.path.exists(attachment['file_path']):
            valid_attachments.append(attachment)
        else:
            print(f"File not found or not accessible: {attachment['filename']}")

    print(f"\nProcessing {len(valid_attachments)} accessible PDF files...")

    # 解析PDF内容
    for idx, attachment in enumerate(valid_attachments):
        try:
            print(f"\n[{idx+1}/{len(valid_attachments)}] Processing: {attachment['filename']}")

            # 打开PDF文件
            doc = fitz.open(attachment['file_path'])

            # 获取基本信息
            print(f"Pages: {len(doc)}")
            print(f"Metadata: {doc.metadata}")

            # 获取文本内容（仅前两页示例）
            max_pages_to_show = min(2, len(doc))
            for page_num in range(max_pages_to_show):
                page = doc[page_num]
                text = page.get_text()
                print(f"\nPage {page_num+1} preview (first 200 chars):")
                print(text[:200] + "..." if len(text) > 200 else text)

            # 关闭文档
            doc.close()

        except Exception as e:
            print(f"Error processing {attachment['filename']}: {str(e)}")

except ModuleNotFoundError:
    print("PyMuPDF (fitz) module not found. Please install it with: pip install pymupdf")
except Exception as e:
    print(f"Error initializing PDF parsing: {str(e)}")



## 构建问题

In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI


# 创建LLM
# 使用OpenAI API
if use_openai:
    llm = ChatOpenAI(
        temperature=0.7,
        model_name=QUESTION_GENERATOR_MODEL,
    )
    embeddings = OpenAIEmbeddings()
# 使用Ollama
else:
    from langchain.llms import Ollama
    llm = Ollama(
        model=QUESTION_GENERATOR_MODEL,  # 或其他你在Ollama中可用的模型
        temperature=0.7
    )
    from langchain.embeddings import OllamaEmbeddings
    embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)

def process_pdf_and_generate_questions(pdf_path):
    print(f"\n处理PDF: {os.path.basename(pdf_path)}")

    # 加载PDF
    loader = PyMuPDFLoader(pdf_path)
    documents = loader.load()
    print(f"已加载 {len(documents)} 页内容")

    # 分割文本
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_documents(documents)
    print(f"文档已分割为 {len(chunks)} 个块")


    vectorstore = FAISS.from_documents(chunks, embeddings)

    # 创建检索器
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    # 创建问答链
    if use_openai:
        from langchain.chat_models import ChatOpenAI
        llm = ChatOpenAI(temperature=0)
    else:
        from langchain.llms import Ollama
        llm = Ollama(model=LLM_MODEL)

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )

    # 生成问题
    prompt = """
# 问答生成

## 背景

上海光源（Shanghai Synchrotron Radiation Facility，简称SSRF）是中国大陆第一台中能第三代同步辐射光源，坐落在浦东张江高科技园区，由国家、中国科学院和上海市人民政府共同投资建设，于2004年12月25日开工建设，2009年4月29日竣工，5月6日正式对用户开放。上海光源首批建成7条光束线站；2015年～2018年期间，"梦之线"、蛋白质设施5线6站、SiP·ME2研究平台等陆续建成。2023年9月，上海光源线站工程顺利通过工艺验收，标志着项目全部建设任务圆满完成，光束线站、用户辅助实验室、加速器等各系统全面转入试运行阶段。目前，上海光源共有34条光束线46个实验站向用户开放运行，总体性能位居国际先进水平。上海光源具有波长范围宽、高强度、高亮度、高准直性、高偏振与准相干性、可准确计算、高稳定性等一系列优异的特性，可用以从事生命科学、材料科学、环境科学、信息科学、凝聚态物理、原子分子物理、团簇物理、化学、医学、药学、地质学等多学科的前沿基础研究，以及微电子、医药、石油、化工、生物工程、医疗诊断和微加工等高技术的开发应用的实验研究。十四年来，上海光源围绕科学前沿、国家重大需求与产业核心问题支撑用户开展创新研究，以工程建设、运行开放、关键技术研发和科学研究四位一体可持续发展的总思路，聚焦重大基础科学突破和关键核心技术发展，为广大用户提供了一个跨学科、综合性、多功能的大科学研究平台，在生命科学、凝聚态物理、材料科学、化学、能源与环境科学等多个学科前沿基础研究和高新技术研发领域产生了一批具有国际影响力的研究成果，有力地推动了相关学科的发展，成为我国科学家参与国际竞争的强大助力。同时，也进一步推动了我国包括同步辐射光源、自由电子激光在内的大科学装置的发展。上海光源是国家重大创新能力基础设施，是支撑众多学科前沿基础研究、应用研究、高新技术研发的大型综合实验研究平台，向各领域用户开放申请。上海光源每年向用户供光4000～5000小时，所有用户均可通过"申请-评审-批准"程序获得上海光源实验机时。目前，中国科学院上海高等研究院负责上海光源的运行、维护和性能升级。

## 任务

你是一个高中生，正在参加上海光源的夏令营。请基于给定的论文内容，生成5-8个你想问上海光源资深工程师的问题。

## 要求

1. 直接输出问题列表，每个问题一行，以数字编号
2. 不要包含任何前言、解释或结尾
3. 问题应该是通用的，不要直接引用论文（不要使用"文中"、"论文中"等字样）
4. 问题应聚焦于上海光源的技术、应用和历史，不要询问未来发展
5. 问题必须使用中文，严禁输出任何英文问题
6. 问题应该具有教育意义，适合高中生的知识水平
7. 问题应该涵盖上海光源的基本原理、应用领域和技术特点
8. 问题应该能引发思考，不要过于简单的是非题

## 示例输出格式
1. 什么是储存环全相干光源？
2. 工作点反馈系统对同步辐射光源有什么作用？
3. 超导波振荡器降温特性是如何进行研究的？
"""
    result = qa_chain({"question": prompt, "chat_history": []})

    print("\n生成的问题:")
    print(result["answer"])

    return result


In [ ]:
# 使用langchain构建agent，基于PDF提出问题
try:
    from langchain.document_loaders import PyMuPDFLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.embeddings import OpenAIEmbeddings
    from langchain.vectorstores import FAISS
    from langchain.chat_models import ChatOpenAI
    from langchain.chains import ConversationalRetrievalChain
    from langchain.agents import initialize_agent, Tool
    from langchain.agents import AgentType
    from langchain.agents.agent import AgentExecutor
    import os

    print("\n初始化LangChain Agent...")


    # 创建工具
    tools = [
        Tool(
            name="PDF问题生成器",
            func=process_pdf_and_generate_questions,
            description="分析PDF文档并生成相关问题"
        )
    ]

    # 初始化agent，添加handle_parsing_errors=True参数
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True
    )

    # 使用示例
    if valid_attachments:
        print("\n使用Agent处理第一个PDF文件...")
        sample_pdf = valid_attachments[0]['file_path']
        agent.invoke(f"分析这个PDF文件并生成问题: {sample_pdf}")
    else:
        print("没有有效的PDF文件可供处理")

except ModuleNotFoundError as e:
    print(f"缺少必要的模块: {e}")
    print("请安装所需模块: pip install langchain openai faiss-cpu pymupdf")
except Exception as e:
    print(f"初始化LangChain Agent时出错: {str(e)}")
    import traceback
    traceback.print_exc()


In [ ]:
# 处理所有PDF并生成问题集
import json
try:
    print("\n处理所有PDF文件并生成问题集...")

    # 添加变量标志是否重新生成问题
    REGENERATE_QUESTIONS = True
    questions_file_path = "./pdf_questions.jsonl"

    all_questions = {}

    # 检查是否存在已保存的问题集且不需要重新生成
    if os.path.exists(questions_file_path) and not REGENERATE_QUESTIONS:
        print(f"从本地文件加载已有问题集: {questions_file_path}")
        # 读取JSONL文件，每行是一个JSON对象
        with open(questions_file_path, 'r', encoding='utf-8') as f:
            all_questions = {}
            for line in f:
                entry = json.loads(line.strip())
                pdf_name = entry.get("pdf_name", "")
                question = entry.get("question", "")
                if pdf_name:
                    if pdf_name not in all_questions:
                        all_questions[pdf_name] = []
                    if question:
                        all_questions[pdf_name].append(question)
        print(f"已加载 {len(all_questions)} 个PDF的问题集")
        # 打印加载的前10个问题
        print("\n前10个问题示例:")
        count = 0
        for pdf_name, questions in all_questions.items():
            for question in questions:
                print(f"- {pdf_name}: {question}")
                count += 1
                if count >= 10:
                    break
            if count >= 10:
                break
    else:
        # 根据DEBUG标志决定处理的PDF数量
        pdf_to_process = valid_attachments[:2] if DEBUG else valid_attachments

        for attachment in pdf_to_process:
            pdf_path = attachment['file_path']
            pdf_name = os.path.basename(pdf_path)

            try:
                print(f"\n正在处理: {pdf_name}")
                result = agent.invoke(f"分析这个PDF文件并生成问题: {pdf_path}")

                # 从结果中提取问题
                if isinstance(result, str):
                    questions_text = result
                else:
                    # 如果agent返回的是字典类型的结果
                    questions_text = result.get('output', "未能提取问题")

                # 拆分问题，每个问题独占一条记录
                questions_list = []
                # 按行分割，然后处理每一行
                for line in questions_text.strip().split('\n'):
                    line = line.strip()
                    # 跳过空行
                    if not line:
                        continue
                    # 跳过可能的标题行、介绍性文字或非问题内容
                    # 使用正则表达式判断是否是以数字+点开头的问题
                    import re
                    # 过滤掉不是以数字+点开头的行，或者是一些介绍性文字
                    if not re.match(r'^\d+\.\s', line) or \
                       "Here are" in line or "以下是" in line or "engineers" in line or "Facility" in line or "The questions are" in line:
                        continue

                    # 如果是有效问题，只保留问题主体，去掉前面的数字编号
                    line = re.sub(r'^\d+\.\s', '', line)
                    # 添加到问题列表
                    questions_list.append(line)

                # 将每个问题单独存储
                all_questions[pdf_name] = questions_list

                print(f"已完成 {pdf_name} 的问题生成，共 {len(questions_list)} 个问题")

            except Exception as e:
                print(f"处理 {pdf_name} 时出错: {str(e)}")
                all_questions[pdf_name] = [f"处理出错: {str(e)}"]

    # 生成JSONL格式，每个问题独占一行
    print(f"\n生成JSONL格式问题集: {questions_file_path}")
    with open(questions_file_path, 'w', encoding='utf-8') as f:
        for pdf_name, questions_list in all_questions.items():
            # 确保questions_list是列表
            if isinstance(questions_list, str):
                questions_list = [questions_list]

            # 为每个问题创建一个单独的JSON对象并写入JSONL文件
            for question in questions_list:
                if question.strip():
                    qa_entry = {
                        "pdf_name": pdf_name,
                        "question": question.strip(),
                        "answer": "",
                        "sources": []
                    }
                    f.write(json.dumps(qa_entry, ensure_ascii=False) + '\n')

    print(f"已将问题集保存为JSONL格式: {questions_file_path}")

except Exception as e:
    print(f"处理PDF文件集时出错: {str(e)}")


## 构建智能体，PDF向量化

In [ ]:
try:
    print("\n构建基于所有PDF的RAG智能体...")

    # 添加向量rebuild标志
    REBUILD_VECTOR = True
    vector_store_path = "./all_pdfs_vectorstore"

    # 创建一个新的向量存储，包含所有有效PDF的内容
    all_pdfs_vectorstore = None

    # 导入re模块
    import re

    # 检查是否需要重建向量存储
    if REBUILD_VECTOR or not os.path.exists(vector_store_path):
        if valid_attachments:
            print("正在构建新的向量存储...")
            # 处理所有PDF文件并合并到一个向量存储中
            all_documents = []
            for attachment in valid_attachments:
                pdf_path = attachment['file_path']
                try:
                    print(f"处理PDF: {pdf_path}")
                    loader = PyMuPDFLoader(pdf_path)
                    documents = loader.load()
                    # 添加元数据以标识来源
                    for doc in documents:
                        doc.metadata["source"] = pdf_path
                    all_documents.extend(documents)
                except Exception as e:
                    print(f"处理PDF时出错 {pdf_path}: {str(e)}")

            # 创建文本分割器
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )

            # 分割文档
            all_splits = text_splitter.split_documents(all_documents)


            # 创建向量存储
            all_pdfs_vectorstore = FAISS.from_documents(all_splits, embeddings)

            # 保存向量存储到本地
            all_pdfs_vectorstore.save_local(vector_store_path)
            print(f"向量存储已保存到: {vector_store_path}")
        else:
            print("没有有效的PDF文件可供处理")
    else:
        print(f"正在加载本地向量存储: {vector_store_path}")
        # 初始化嵌入模型        # 从本地加载向量存储，添加allow_dangerous_deserialization=True参数
        all_pdfs_vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
        print("本地向量存储加载完成")

    if all_pdfs_vectorstore:
        # 创建检索器
        all_pdfs_retriever = all_pdfs_vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5}
        )

        # 创建问答链
        all_pdfs_qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=all_pdfs_retriever,
            return_source_documents=True
        )

except Exception as e:
    print(f"构建RAG智能体时出错: {str(e)}")
    import traceback
    traceback.print_exc()


## 生成问题答案

In [ ]:
# 使用RAG生成回答并保存为jsonl格式
try:
    import json

    # 定义输出文件
    qa_output_file = "pdf_qa_results.jsonl"

    if all_pdfs_qa_chain and all_questions:
        print("\n回答生成的问题并保存为JSONL格式:")

        # 打开文件准备写入
        with open(qa_output_file, 'w', encoding='utf-8') as f:
            # 遍历每个PDF文件的问题
            for pdf_name, questions_text in all_questions.items():
                print(f"\n处理来自 {pdf_name} 的问题:")

                # 尝试将问题文本分割成单独的问题
                questions = questions_text.split("\n")

                for question in questions:
                    # 清理问题文本
                    clean_question = re.sub(r'^\d+\.\s*', '', question).strip()
                    if clean_question and len(clean_question) > 10:  # 确保问题有足够长度
                        print(f"\n问题: {clean_question}")
                        answer_result = all_pdfs_qa_chain.invoke({"question": clean_question, "chat_history": []})
                        print(f"回答: {answer_result['answer']}")

                        # 显示来源
                        print("来源文档:")
                        sources = set()
                        for doc in answer_result["source_documents"]:
                            if "source" in doc.metadata:
                                sources.add(doc.metadata["source"])
                        for source in sources:
                            print(f"- {source}")

                        # 创建要保存的记录
                        qa_record = {
                            "pdf_name": pdf_name,
                            "question": clean_question,
                            "answer": answer_result['answer'],
                            "sources": list(sources)
                        }

                        # 写入JSONL文件
                        f.write(json.dumps(qa_record, ensure_ascii=False) + '\n')

        print(f"\n问答记录已保存到: {qa_output_file}")
    else:
        print("无法生成回答：RAG智能体未创建或没有找到之前生成的问题")

except Exception as e:
    print(f"生成回答时出错: {str(e)}")
    import traceback
    traceback.print_exc()

## 补全推理过程

In [ ]:
# 构建Agent来补全推理过程
try:
    from langchain.chat_models import ChatOpenAI
    from langchain.schema import HumanMessage, SystemMessage
    from langchain.prompts import ChatPromptTemplate
    import json
    import os

    # 定义输入和输出文件
    qa_input_file = "pdf_qa_results.jsonl"  # 之前保存的问答结果文件
    reasoning_output_file = "qa_with_reasoning.jsonl"  # 包含推理过程的输出文件

    # 检查输入文件是否存在
    if not os.path.exists(qa_input_file):
        print(f"错误：找不到输入文件 {qa_input_file}")
    else:
        # 初始化大模型
        llm = ChatOpenAI(
            temperature=0.1,
            model_name="deepseek-ai/DeepSeek-R1"
            )

        # 创建提示模板
        system_message = """你是一个专业的学术助手，精通各学科领域的专业知识。你的任务是为给定的问题和答案补充详细、准确的推理过程，展示从问题到答案的完整思考路径。

请注意以下要求：
1. 推理过程必须清晰展示从问题分析到得出答案的完整思考链条
2. 基于可靠的学术原理和专业知识，不要编造信息
3. 包含具体的技术细节、公式、原理和机制解释，而非泛泛而谈
4. 使用该领域的专业术语和标准表达方式
5. 按照逻辑顺序组织内容，可以使用编号或小标题增强结构性
6. 必要时引入定量分析，包括数值范围、单位和量级
7. 解释关键参数之间的关系和影响机制
8. 明确说明每一步推理如何引导到下一步，最终如何得出答案中的结论

推理过程应该结构清晰，包含以下部分：
- 问题分析：明确问题的核心要点和需要解决的关键问题
- 技术原理：相关的专业知识和技术原理
- 工作机制：关键组件的工作机制和参数关系
- 系统设计：系统设计中的关键考量和技术路线
- 性能分析：系统性能指标和应用价值

避免使用"XX"这样的占位符，确保所有内容都是具体、明确的。不要使用模板化的结构，而是根据具体问题提供有针对性的分析。"""

        # 打开输入文件读取问答记录
        qa_records = []
        with open(qa_input_file, 'r', encoding='utf-8') as f:
            for line in f:
                qa_records.append(json.loads(line.strip()))

        print(f"读取了 {len(qa_records)} 条问答记录")

        # 如果DEBUG模式，只处理第一条记录
        if DEBUG:
            qa_records = qa_records[:1]
            print("DEBUG模式：只处理第一条记录")

        # 打开输出文件准备写入
        with open(reasoning_output_file, 'w', encoding='utf-8') as f:
            # 遍历每条问答记录
            for i, record in enumerate(qa_records):
                print(f"\n处理第 {i+1}/{len(qa_records)} 条记录...")

                # 获取问题和答案
                question = record["question"]
                answer = record["answer"]

                print(f"问题: {question[:50]}..." if len(question) > 50 else f"问题: {question}")
                print(f"答案: {answer[:100]}..." if len(answer) > 100 else f"答案: {answer}")

                # 构建提示并调用大模型
                human_message = f"""问题: {question}

答案: {answer}

请为上述问答补充详细的推理过程，确保清晰展示从问题到答案的完整思考路径，包含足够的专业深度和技术细节。"""

                messages = [
                    SystemMessage(content=system_message),
                    HumanMessage(content=human_message)
                ]

                # 打印提示信息
                print("发送到LLM的提示:")
                for msg in messages:
                    print(f"[{msg.type}]: {msg.content[:100]}..." if len(msg.content) > 100 else f"[{msg.type}]: {msg.content}")
                print("-" * 50)

                # 获取推理过程
                response = llm.invoke(messages)
                reasoning = response.content

                # 更新记录，添加推理过程
                record["reasoning"] = reasoning

                # 写入JSONL文件
                f.write(json.dumps(record, ensure_ascii=False) + '\n')

                # 打印推理过程
                print(f"推理过程:\n{reasoning}")
                print(f"已添加推理过程 ({len(reasoning)} 字符)")

        print(f"\n推理过程已添加并保存到: {reasoning_output_file}")
        if DEBUG:
            print("注意：由于DEBUG模式开启，只处理了第一条记录")

except Exception as e:
    print(f"补全推理过程时出错: {str(e)}")
    import traceback
    traceback.print_exc()



# Zotero PDF生成大模型训练数据集

## 安装依赖

### 安装 PyMuPDF
```shell
pip install PyMuPDF
```

### 安装Pyzotero
```shell
pip install pyzotero
# 6zWcYYQ2eiVYDUokDh1RsI0R
```

### 安装 langchain & openai
```shell
pip install openai langchain langchain-community tiktoken faiss-cpu
```

## 初始化KEY

In [1]:
import os

os.environ["OPENAI_API_KEY"] = ''
os.environ["OPENAI_BASE_URL"] = 'https://aihubmix.com/v1'

use_openai = True
QUESTION_GENERATOR_MODEL = "aihubmix-Llama-3-3-70B-Instruct"
ANSWER_GENERATOR_MODEL = "aihubmix-Llama-3-3-70B-Instruct"
THINKING_MODEL = "aihubmix-Llama-3-3-70B-Instruct"


In [58]:
import os
# Remove OPENAI_API_KEY from environment variables
if "OPENAI_API_KEY" in os.environ:
    del os.environ["OPENAI_API_KEY"]
    
# Set Ollama base URL for local usage
os.environ["OPENAI_BASE_URL"] = "http://localhost:11434"

use_openai = False
EMBEDDING_MODEL = "deepseek-r1:70b"
QUESTION_GENERATOR_MODEL = "deepseek-r1:70b"
ANSWER_GENERATOR_MODEL = "deepseek-r1:70b"
THINKING_MODEL = "deepseek-r1:70b"


In [59]:
DEBUG = True

## 读取zotero的信息

In [60]:
from pyzotero import zotero

zot = zotero.Zotero(
    library_id='15307681',
    library_type='user',
    local=True
)

# 获取所有collections
collections = zot.collections()

# 查找"同步辐射光源"分类
target_collection = None
for collection in collections:
    if collection['data']['name'] == '同步辐射光源':
        target_collection = collection
        break

if  not target_collection:
    print("未找到'同步辐射光源'分类")

    # 获取该分类下的所有条目
items = zot.collection_items(target_collection['key'])
    
    # 打印分类信息和条目信息
print(f"Collection: {target_collection['data']['name']}")
print(f"Collection Key: {target_collection['key']}")
print(f"Collection Items: {len(items)}")
print("\nItems in collection:")
for item in items:
    print(f"\nTitle: {item['data']['title']}")
    print(f"Type: {item['data']['itemType']}")
    if 'abstractNote' in item['data']:
        print(f"Abstract: {item['data']['abstractNote']}")


Collection: 同步辐射光源
Collection Key: ZY8SEM2W
Collection Items: 91

Items in collection:

Title: 上海光源X射线针孔相机控制系统设计
Type: journalArticle
Abstract: 采用美国NI公司的PXI-7344运动控制卡对衰减片进行控制;通过RS232协议控制小孔四维移动平台和相机二维移动平台的运动,实现对小孔位置、倾角和相机位置的调节;NI公司的PXI-8252板卡用于1394相机的图像采集。所有控制功能均有机地集成于LabVIEW界面中,实现各部件的协调配合运动,并通过EPICS Shared Memory IOCcore技术实现与上海光源控制系统的无缝连接。2009年8月开始的长时间在线运行证明该控制系统稳定可靠,可在线实时测量数十μm束团横向截面尺寸,实现nm弧度发射度测量。

Title: 基于EPICS的上海光源注入引出远控系统
Type: journalArticle
Abstract: 基于EPICS的应用开发,实现了上海光源注入引出系统的远程控制。介绍了控制系统的硬件结构及其工作原理,阐述了远程控制系统的组成及其设计方法。运行结果表明,控制精度高、占用资源少、灵敏度高、工作稳定可靠,满足了控制功能的需求。

Title: 上海光源直线加速器的弱流强运行
Type: journalArticle
Abstract: 上海光源(SSRF)直线加速器正常运行时,重复频率2 Hz,单束团脉宽1 ns,电荷量1 nC。近年来,国际同步辐射领域发展起来的恒流注入模式要求直线加速器以弱流强运行,在相同的重复频率和脉宽下,单脉冲电荷量越小越好,此方法重复性较好。本文在理论计算基础上,结合实验,给出了弱流强稳定运行的实现方法,确定了在Top-up模式下,直线加速器的脉冲电荷量为0.1 nC。

Title: 上海光源调制器远控系统的设计与实现
Type: journalArticle
Abstract: 介绍了在EPICS环境下设计和实现对上海光源直线调制器的远程控制。阐述了系统结构、工作原理、硬件结构、软件设计,描述了远控系统的功能及实现方法。该系统具有人机交互性好、操作使用简便等优点,取得了良好的监控效果。

Title: 上海光源XBPM自动量程I-

## 获取所有附件信息

In [61]:
# 获取所有条目的附件信息
import urllib


attachments = []
print(f"Total items: {len(items)}")

for item in items:
    # 获取每个条目的附件
    item_attachments = zot.children(item['key'])
    for attachment in item_attachments:
        if attachment['data']['itemType'] == 'attachment' and 'contentType' in attachment['data']:
            # 只处理PDF附件
            if attachment['data']['contentType'] == 'application/pdf':
                # 从enclosure链接中获取本地文件路径
                file_path = urllib.parse.unquote(attachment['links']['enclosure']['href'].replace('file:///', '')) if 'enclosure' in attachment['links'] else None
                
                attachment_info = {
                    'parent_title': item['data']['title'],  # 父条目标题
                    'parent_key': item['key'],              # 父条目key
                    'attachment_key': attachment['key'],    # 附件key
                    'filename': attachment['data']['filename'],  # 文件名
                    'file_path': file_path,                # 本地文件路径
                    'content_type': attachment['data']['contentType'],  # 内容类型
                    'added_date': attachment['data']['dateAdded'],     # 添加日期
                    'modified_date': attachment['data']['dateModified'],# 修改日期
                    'md5': attachment['data'].get('md5', ''),         # MD5校验和
                }
                attachments.append(attachment_info)

print(f"\nFound {len(attachments)} PDF attachments:")
for attachment in attachments:
    print(f"\nParent Item: {attachment['parent_title']}")
    print(f"Filename: {attachment['filename']}")
    print(f"File Path: {attachment['file_path']}")
    print(f"Added Date: {attachment['added_date']}")
    print(f"MD5: {attachment['md5']}")


Total items: 91

Found 91 PDF attachments:

Parent Item: 上海光源X射线针孔相机控制系统设计
Filename: 黄国庆 等 - 2011 - 上海光源X射线针孔相机控制系统设计.pdf
File Path: C:/Users/fly/Zotero/storage/YEQVL53Q/黄国庆 等 - 2011 - 上海光源X射线针孔相机控制系统设计.pdf
Added Date: 2025-02-24T02:38:42Z
MD5: dfa7e68226e0dfb25dd7067c54ab9a2e

Parent Item: 基于EPICS的上海光源注入引出远控系统
Filename: 朱海君 等 - 2012 - 基于EPICS的上海光源注入引出远控系统.pdf
File Path: C:/Users/fly/Zotero/storage/WSVXXARB/朱海君 等 - 2012 - 基于EPICS的上海光源注入引出远控系统.pdf
Added Date: 2025-02-24T02:37:26Z
MD5: de5a327986c148c9a1c275cf736fec2c

Parent Item: 上海光源直线加速器的弱流强运行
Filename: 汪宝亮 等 - 2012 - 上海光源直线加速器的弱流强运行.pdf
File Path: C:/Users/fly/Zotero/storage/ZKRTTWUN/汪宝亮 等 - 2012 - 上海光源直线加速器的弱流强运行.pdf
Added Date: 2025-02-24T02:37:05Z
MD5: e8283ae053a0ffad823bf4721dabce86

Parent Item: 上海光源调制器远控系统的设计与实现
Filename: 朱海君 等 - 2012 - 上海光源调制器远控系统的设计与实现.pdf
File Path: C:/Users/fly/Zotero/storage/JB3ND9Q4/朱海君 等 - 2012 - 上海光源调制器远控系统的设计与实现.pdf
Added Date: 2025-02-24T02:36:45Z
MD5: a0ba6ff438e85da46373a28a1f287554

Parent Item: 

## 解析PDF信息

In [62]:
# 解析PDF信息
try:
    import fitz # PyMuPDF
    import os
    
    print("\nParsing PDF contents...")
    
    # 检查PDF文件是否存在并可访问
    valid_attachments = []
    for attachment in attachments:
        if attachment['file_path'] and os.path.exists(attachment['file_path']):
            valid_attachments.append(attachment)
        else:
            print(f"File not found or not accessible: {attachment['filename']}")
    
    print(f"\nProcessing {len(valid_attachments)} accessible PDF files...")
    
    # 解析PDF内容
    for idx, attachment in enumerate(valid_attachments):
        try:
            print(f"\n[{idx+1}/{len(valid_attachments)}] Processing: {attachment['filename']}")
            
            # 打开PDF文件
            doc = fitz.open(attachment['file_path'])
            
            # 获取基本信息
            print(f"Pages: {len(doc)}")
            print(f"Metadata: {doc.metadata}")
            
            # 获取文本内容（仅前两页示例）
            max_pages_to_show = min(2, len(doc))
            for page_num in range(max_pages_to_show):
                page = doc[page_num]
                text = page.get_text()
                print(f"\nPage {page_num+1} preview (first 200 chars):")
                print(text[:200] + "..." if len(text) > 200 else text)
            
            # 关闭文档
            doc.close()
            
        except Exception as e:
            print(f"Error processing {attachment['filename']}: {str(e)}")
    
except ModuleNotFoundError:
    print("PyMuPDF (fitz) module not found. Please install it with: pip install pymupdf")
except Exception as e:
    print(f"Error initializing PDF parsing: {str(e)}")




Parsing PDF contents...

Processing 91 accessible PDF files...

[1/91] Processing: 黄国庆 等 - 2011 - 上海光源X射线针孔相机控制系统设计.pdf
Pages: 5
Metadata: {'format': 'PDF 1.6', 'title': '', 'author': 'CNKI', 'subject': '', 'keywords': '', 'creator': 'ReaderEx_DIS 2.5.0 Build 4087', 'producer': 'TTKN', 'creationDate': "D:20241013182522-08'00'", 'modDate': '', 'trapped': '', 'encryption': None}

Page 1 preview (first 200 chars):
第34 卷 第9 期                                核    技    术                     
Vol. 34, No.9 
2011 年9 月 
NUCLEAR TECHNIQUES 
September 2011 
—————————————— 
第一作者：黄国庆，男，1977 年出生，2007 年于中国科学院上海应用物理研究所获博士学位，...

Page 2 preview (first 200 chars):
642 
核    技    术 
第34 卷 
 
PXI-8252 板卡有3 个1394 接口，可接3 个1394 设
备。通过图像采集LabVIEW 模块，可方便采集X
射线图像。 
1.2 
小孔位置控制 
该X 射线针孔相机系统的小孔模块由9 种尺寸
的小孔阵列组成[3]。小孔阵列由水平移动、垂直移
动、水平旋转、俯仰旋转精密移动平台控制。在
LabVIEW 中有9 个孔对应的控键，点击...

[2/91] Processing: 朱海君 等 - 2012 - 基于EPICS的上海光源注入引出远控系统.pdf
Pages: 4
Metadata: {'format': 'PDF 1.6', 'title': '', 'author': 'CNKI', 'subject'

## 构建问题

In [67]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI


# 创建LLM
# 使用OpenAI API
if use_openai:
    llm = ChatOpenAI(
        temperature=0.7, 
        model_name=QUESTION_GENERATOR_MODEL,
    )
    embeddings = OpenAIEmbeddings()
# 使用Ollama
else:
    from langchain.llms import Ollama
    llm = Ollama(
        model=QUESTION_GENERATOR_MODEL,  # 或其他你在Ollama中可用的模型
        temperature=0.7
    )
    from langchain.embeddings import OllamaEmbeddings
    embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)

def process_pdf_and_generate_questions(pdf_path):
    print(f"\n处理PDF: {os.path.basename(pdf_path)}")
    
    # 加载PDF
    loader = PyMuPDFLoader(pdf_path)
    documents = loader.load()
    print(f"已加载 {len(documents)} 页内容")
    
    # 分割文本
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_documents(documents)
    print(f"文档已分割为 {len(chunks)} 个块")

    
    vectorstore = FAISS.from_documents(chunks, embeddings)
    
    # 创建检索器
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    
    # 创建问答链
    if use_openai:
        from langchain.chat_models import ChatOpenAI
        llm = ChatOpenAI(temperature=0)
    else:
        from langchain.llms import Ollama
        llm = Ollama(model=LLM_MODEL)
        
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )
    
    # 生成问题
    prompt = """
# 问答生成

## 背景

上海光源（Shanghai Synchrotron Radiation Facility，简称SSRF）是中国大陆第一台中能第三代同步辐射光源，坐落在浦东张江高科技园区，由国家、中国科学院和上海市人民政府共同投资建设，于2004年12月25日开工建设，2009年4月29日竣工，5月6日正式对用户开放。上海光源首批建成7条光束线站；2015年～2018年期间，"梦之线"、蛋白质设施5线6站、SiP·ME2研究平台等陆续建成。2023年9月，上海光源线站工程顺利通过工艺验收，标志着项目全部建设任务圆满完成，光束线站、用户辅助实验室、加速器等各系统全面转入试运行阶段。目前，上海光源共有34条光束线46个实验站向用户开放运行，总体性能位居国际先进水平。上海光源具有波长范围宽、高强度、高亮度、高准直性、高偏振与准相干性、可准确计算、高稳定性等一系列优异的特性，可用以从事生命科学、材料科学、环境科学、信息科学、凝聚态物理、原子分子物理、团簇物理、化学、医学、药学、地质学等多学科的前沿基础研究，以及微电子、医药、石油、化工、生物工程、医疗诊断和微加工等高技术的开发应用的实验研究。十四年来，上海光源围绕科学前沿、国家重大需求与产业核心问题支撑用户开展创新研究，以工程建设、运行开放、关键技术研发和科学研究四位一体可持续发展的总思路，聚焦重大基础科学突破和关键核心技术发展，为广大用户提供了一个跨学科、综合性、多功能的大科学研究平台，在生命科学、凝聚态物理、材料科学、化学、能源与环境科学等多个学科前沿基础研究和高新技术研发领域产生了一批具有国际影响力的研究成果，有力地推动了相关学科的发展，成为我国科学家参与国际竞争的强大助力。同时，也进一步推动了我国包括同步辐射光源、自由电子激光在内的大科学装置的发展。上海光源是国家重大创新能力基础设施，是支撑众多学科前沿基础研究、应用研究、高新技术研发的大型综合实验研究平台，向各领域用户开放申请。上海光源每年向用户供光4000～5000小时，所有用户均可通过"申请-评审-批准"程序获得上海光源实验机时。目前，中国科学院上海高等研究院负责上海光源的运行、维护和性能升级。

## 任务

你是一个高中生，正在参加上海光源的夏令营。请基于给定的论文内容，生成5-8个你想问上海光源资深工程师的问题。

## 要求

1. 直接输出问题列表，每个问题一行，以数字编号
2. 不要包含任何前言、解释或结尾
3. 问题应该是通用的，不要直接引用论文（不要使用"文中"、"论文中"等字样）
4. 问题应聚焦于上海光源的技术、应用和历史，不要询问未来发展
5. 问题必须使用中文，严禁输出任何英文问题
6. 问题应该具有教育意义，适合高中生的知识水平
7. 问题应该涵盖上海光源的基本原理、应用领域和技术特点
8. 问题应该能引发思考，不要过于简单的是非题

## 示例输出格式
1. 什么是储存环全相干光源？
2. 工作点反馈系统对同步辐射光源有什么作用？
3. 超导波振荡器降温特性是如何进行研究的？
"""
    result = qa_chain({"question": prompt, "chat_history": []})
    
    print("\n生成的问题:")
    print(result["answer"])
    
    return result


In [64]:
# 使用langchain构建agent，基于PDF提出问题
try:
    from langchain.document_loaders import PyMuPDFLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.embeddings import OpenAIEmbeddings
    from langchain.vectorstores import FAISS
    from langchain.chat_models import ChatOpenAI
    from langchain.chains import ConversationalRetrievalChain
    from langchain.agents import initialize_agent, Tool
    from langchain.agents import AgentType
    from langchain.agents.agent import AgentExecutor
    import os
    
    print("\n初始化LangChain Agent...")

        
    # 创建工具
    tools = [
        Tool(
            name="PDF问题生成器",
            func=process_pdf_and_generate_questions,
            description="分析PDF文档并生成相关问题"
        )
    ]
    
    # 初始化agent，添加handle_parsing_errors=True参数
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True
    )
    
    # 使用示例
    if valid_attachments:
        print("\n使用Agent处理第一个PDF文件...")
        sample_pdf = valid_attachments[0]['file_path']
        agent.invoke(f"分析这个PDF文件并生成问题: {sample_pdf}")
    else:
        print("没有有效的PDF文件可供处理")

except ModuleNotFoundError as e:
    print(f"缺少必要的模块: {e}")
    print("请安装所需模块: pip install langchain openai faiss-cpu pymupdf")
except Exception as e:
    print(f"初始化LangChain Agent时出错: {str(e)}")
    import traceback
    traceback.print_exc()



初始化LangChain Agent...

使用Agent处理第一个PDF文件...


> Entering new AgentExecutor chain...
<think>
嗯，我现在需要分析一下这个PDF文件，然后根据内容生成一些相关的问题。首先，文件的路径是C:/Users/fly/Zotero/storage/YEQVL53Q/黄国庆 等 - 2011 - 上海光源X射线针孔相机控制系统设计.pdf，看起来这是一个关于上海光源X射线针孔相机控制系统设计的论文或者报告，作者包括黄国庆等人，发表于2011年。

首先，我应该理解这篇文章的主要内容。这可能涉及到X射线针孔相机的工作原理、系统架构、控制系统的设计细节，比如硬件和软件部分，以及在上海光源中的应用情况。因此，我需要生成的问题应该涵盖这些方面。

接下来，我可以想到几个大致的问题类别：系统概述、核心组件、技术细节、实际应用以及面临的挑战或解决方案。每个类别下，可以进一步细化问题，比如询问设计目标、所使用的具体技术（比如CCD相机）、控制系统的实现方式等。

然后，考虑到文章可能会提到的具体内容，比如X射线检测的原理，针孔相机的结构与工作原理，以及如何将这些整合到上海光源的环境中。此外，还可能涉及数据采集、触发模式、同步控制等详细的技术点。

我还需要考虑系统设计中的创新点，比如是否提出了新的算法或硬件架构，或者在实际应用中遇到了哪些挑战，并如何克服这些困难。这些建议可以帮助生成更深入的问题，有助于全面理解论文内容。

此外，我应该确保问题涵盖理论和实践的结合，比如系统设计与实现过程中的具体步骤，以及实验结果的分析。这样可以为读者提供全面的视角，了解研究的深度和广度。

最后，要注意生成的问题具有可操作性，即每个问题都应明确，并且能够通过阅读论文内容找到答案。这意味着问题需要具体而不泛泛，同时覆盖论文的不同部分，从概述到详细技术点，再到应用和挑战。
</think>

好的，我已经分析了PDF文件，并根据其内容生成了一些相关的问题。这些问题涵盖了系统设计、核心组件、技术细节以及实际应用等方面，旨在全面了解论文的主要内容。

以下是根据PDF内容生成的一些问题：

1. 上海光源X射线针孔相机控制系统的整体架构是什么？
2. 系统的主要设计目标和性能指标有哪些？
3. X射线针孔相机的核心组件包括哪

Traceback (most recent call last):
  File "C:\Users\fly\AppData\Local\Temp\ipykernel_13748\2478749492.py", line 39, in <module>
    agent.invoke(f"分析这个PDF文件并生成问题: {sample_pdf}")
  File "d:\Dev\LLMExpertSystem\.venv\lib\site-packages\langchain\chains\base.py", line 170, in invoke
    raise e
  File "d:\Dev\LLMExpertSystem\.venv\lib\site-packages\langchain\chains\base.py", line 160, in invoke
    self._call(inputs, run_manager=run_manager)
  File "d:\Dev\LLMExpertSystem\.venv\lib\site-packages\langchain\agents\agent.py", line 1624, in _call
    next_step_output = self._take_next_step(
  File "d:\Dev\LLMExpertSystem\.venv\lib\site-packages\langchain\agents\agent.py", line 1330, in _take_next_step
    [
  File "d:\Dev\LLMExpertSystem\.venv\lib\site-packages\langchain\agents\agent.py", line 1330, in <listcomp>
    [
  File "d:\Dev\LLMExpertSystem\.venv\lib\site-packages\langchain\agents\agent.py", line 1415, in _iter_next_step
    yield self._perform_agent_action(
  File "d:\Dev\LLMExpertSy

In [57]:
# 处理所有PDF并生成问题集
import json
try:
    print("\n处理所有PDF文件并生成问题集...")
    
    # 添加变量标志是否重新生成问题
    REGENERATE_QUESTIONS = True
    questions_file_path = "./pdf_questions.jsonl"
    
    all_questions = {}
    
    # 检查是否存在已保存的问题集且不需要重新生成
    if os.path.exists(questions_file_path) and not REGENERATE_QUESTIONS:
        print(f"从本地文件加载已有问题集: {questions_file_path}")
        # 读取JSONL文件，每行是一个JSON对象
        with open(questions_file_path, 'r', encoding='utf-8') as f:
            all_questions = {}
            for line in f:
                entry = json.loads(line.strip())
                pdf_name = entry.get("pdf_name", "")
                question = entry.get("question", "")
                if pdf_name:
                    if pdf_name not in all_questions:
                        all_questions[pdf_name] = []
                    if question:
                        all_questions[pdf_name].append(question)
        print(f"已加载 {len(all_questions)} 个PDF的问题集")
        # 打印加载的前10个问题
        print("\n前10个问题示例:")
        count = 0
        for pdf_name, questions in all_questions.items():
            for question in questions:
                print(f"- {pdf_name}: {question}")
                count += 1
                if count >= 10:
                    break
            if count >= 10:
                break
    else:
        # 根据DEBUG标志决定处理的PDF数量
        pdf_to_process = valid_attachments[:2] if DEBUG else valid_attachments
        
        for attachment in pdf_to_process:
            pdf_path = attachment['file_path']
            pdf_name = os.path.basename(pdf_path)
            
            try:
                print(f"\n正在处理: {pdf_name}")
                result = agent.invoke(f"分析这个PDF文件并生成问题: {pdf_path}")
                
                # 从结果中提取问题
                if isinstance(result, str):
                    questions_text = result
                else:
                    # 如果agent返回的是字典类型的结果
                    questions_text = result.get('output', "未能提取问题")
                
                # 拆分问题，每个问题独占一条记录
                questions_list = []
                # 按行分割，然后处理每一行
                for line in questions_text.strip().split('\n'):
                    line = line.strip()
                    # 跳过空行
                    if not line:
                        continue
                    # 跳过可能的标题行、介绍性文字或非问题内容
                    # 使用正则表达式判断是否是以数字+点开头的问题
                    import re
                    # 过滤掉不是以数字+点开头的行，或者是一些介绍性文字
                    if not re.match(r'^\d+\.\s', line) or \
                       "Here are" in line or "以下是" in line or "engineers" in line or "Facility" in line or "The questions are" in line:
                        continue
                    
                    # 如果是有效问题，只保留问题主体，去掉前面的数字编号
                    line = re.sub(r'^\d+\.\s', '', line)
                    # 添加到问题列表
                    questions_list.append(line)
                
                # 将每个问题单独存储
                all_questions[pdf_name] = questions_list
                    
                print(f"已完成 {pdf_name} 的问题生成，共 {len(questions_list)} 个问题")
                
            except Exception as e:
                print(f"处理 {pdf_name} 时出错: {str(e)}")
                all_questions[pdf_name] = [f"处理出错: {str(e)}"]
    
    # 生成JSONL格式，每个问题独占一行
    print(f"\n生成JSONL格式问题集: {questions_file_path}")
    with open(questions_file_path, 'w', encoding='utf-8') as f:
        for pdf_name, questions_list in all_questions.items():
            # 确保questions_list是列表
            if isinstance(questions_list, str):
                questions_list = [questions_list]
            
            # 为每个问题创建一个单独的JSON对象并写入JSONL文件
            for question in questions_list:
                if question.strip():
                    qa_entry = {
                        "pdf_name": pdf_name,
                        "question": question.strip(),
                        "answer": "",
                        "sources": []
                    }
                    f.write(json.dumps(qa_entry, ensure_ascii=False) + '\n')
    
    print(f"已将问题集保存为JSONL格式: {questions_file_path}")
    
except Exception as e:
    print(f"处理PDF文件集时出错: {str(e)}")



处理所有PDF文件并生成问题集...

正在处理: 黄国庆 等 - 2011 - 上海光源X射线针孔相机控制系统设计.pdf


> Entering new AgentExecutor chain...
Thought: 我需要使用PDF问题生成器工具来分析指定的PDF文件并生成相关问题。

Action: PDF问题生成器
Action Input: C:/Users/fly/Zotero/storage/YEQVL53Q/黄国庆 等 - 2011 - 上海光源X射线针孔相机控制系统设计.pdf
处理PDF: 黄国庆 等 - 2011 - 上海光源X射线针孔相机控制系统设计.pdf
已加载 5 页内容
文档已分割为 10 个块

生成的问题:
1. 上海光源的X射线针孔相机控制系统是如何设计的？
2. 为什么X射线成像系统在测量束团尺寸方面比可见光系统更优越？
3. 上海光源的X射线针孔相机系统如何与EPICS控制系统无缝连接的？
4. 什么是EPICS接口，它在上海光源的控制系统中扮演着怎样的角色？
5. 如何在上海光源中实现对束团尺寸等数据的不间断保存？
6. X射线针孔相机系统如何帮助测量SSRF的束团发射度？
7. 为什么使用X射线针孔相机系统可以提高束团尺寸的分辨率？

Observation: {'question': '\n# 问答生成\n\n## 背景\n\n上海光源（Shanghai Synchrotron Radiation Facility，简称SSRF）是中国大陆第一台中能第三代同步辐射光源，坐落在浦东张江高科技园区，由国家、中国科学院和上海市人民政府共同投资建设，于2004年12月25日开工建设，2009年4月29日竣工，5月6日正式对用户开放。上海光源首批建成7条光束线站；2015年～2018年期间，"梦之线"、蛋白质设施5线6站、SiP·ME2研究平台等陆续建成。2023年9月，上海光源线站工程顺利通过工艺验收，标志着项目全部建设任务圆满完成，光束线站、用户辅助实验室、加速器等各系统全面转入试运行阶段。目前，上海光源共有34条光束线46个实验站向用户开放运行，总体性能位居国际先进水平。上海光源具有波长范围宽、高强度、高亮度、高准直性、高偏振与准相干性、可准确计算、高稳定性等一系列优异的特性，可用以从事生命科学、材料科学、环境科学、

## 构建智能体，PDF向量化

In [69]:
try:
    print("\n构建基于所有PDF的RAG智能体...")
    
    # 添加向量rebuild标志
    REBUILD_VECTOR = True
    vector_store_path = "./all_pdfs_vectorstore"
    
    # 创建一个新的向量存储，包含所有有效PDF的内容
    all_pdfs_vectorstore = None
    
    # 导入re模块
    import re
    
    # 检查是否需要重建向量存储
    if REBUILD_VECTOR or not os.path.exists(vector_store_path):
        if valid_attachments:
            print("正在构建新的向量存储...")
            # 处理所有PDF文件并合并到一个向量存储中
            all_documents = []
            for attachment in valid_attachments:
                pdf_path = attachment['file_path']
                try:
                    print(f"处理PDF: {pdf_path}")
                    loader = PyMuPDFLoader(pdf_path)
                    documents = loader.load()
                    # 添加元数据以标识来源
                    for doc in documents:
                        doc.metadata["source"] = pdf_path
                    all_documents.extend(documents)
                except Exception as e:
                    print(f"处理PDF时出错 {pdf_path}: {str(e)}")
            
            # 创建文本分割器
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )
            
            # 分割文档
            all_splits = text_splitter.split_documents(all_documents)

            
            # 创建向量存储
            all_pdfs_vectorstore = FAISS.from_documents(all_splits, embeddings)
            
            # 保存向量存储到本地
            all_pdfs_vectorstore.save_local(vector_store_path)
            print(f"向量存储已保存到: {vector_store_path}")
        else:
            print("没有有效的PDF文件可供处理")
    else:
        print(f"正在加载本地向量存储: {vector_store_path}")
        # 初始化嵌入模型        # 从本地加载向量存储，添加allow_dangerous_deserialization=True参数
        all_pdfs_vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
        print("本地向量存储加载完成")
    
    if all_pdfs_vectorstore:
        # 创建检索器
        all_pdfs_retriever = all_pdfs_vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5}
        )
        
        # 创建问答链
        all_pdfs_qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=all_pdfs_retriever,
            return_source_documents=True
        )
        
except Exception as e:
    print(f"构建RAG智能体时出错: {str(e)}")
    import traceback
    traceback.print_exc()



构建基于所有PDF的RAG智能体...
正在构建新的向量存储...
处理PDF: C:/Users/fly/Zotero/storage/YEQVL53Q/黄国庆 等 - 2011 - 上海光源X射线针孔相机控制系统设计.pdf
处理PDF: C:/Users/fly/Zotero/storage/WSVXXARB/朱海君 等 - 2012 - 基于EPICS的上海光源注入引出远控系统.pdf
处理PDF: C:/Users/fly/Zotero/storage/ZKRTTWUN/汪宝亮 等 - 2012 - 上海光源直线加速器的弱流强运行.pdf
处理PDF: C:/Users/fly/Zotero/storage/JB3ND9Q4/朱海君 等 - 2012 - 上海光源调制器远控系统的设计与实现.pdf
处理PDF: C:/Users/fly/Zotero/storage/BVCHYDR2/张永立 等 - 2012 - 上海光源XBPM自动量程I-V转换器的研制.pdf
处理PDF: C:/Users/fly/Zotero/storage/SZ8846E8/肖体乔 等 - 2014 - 上海光源X射线成像及其应用研究进展.pdf
处理PDF: C:/Users/fly/Zotero/storage/UC3U6WI5/耿合龙 等 - 2015 - 上海光源储存环束流轨道联锁系统升级.pdf
处理PDF: C:/Users/fly/Zotero/storage/JSDVYY3E/张永立 等 - 2014 - 上海光源红外光束线主动反馈控制器的设计.pdf
处理PDF: C:/Users/fly/Zotero/storage/ZFVQJV4J/朱周侠和龚培荣 - 2014 - 上海光源Canted光束线安全联锁系统设计.pdf
处理PDF: C:/Users/fly/Zotero/storage/7UXC4WDH/张招红 等 - 2015 - 上海光源XAFS线站时间分辨X射线激发发光谱实验系统.pdf
处理PDF: C:/Users/fly/Zotero/storage/BEUYNDXY/殷重先 等 - 2015 - 上海光源快速轨道反馈系统.pdf
处理PDF: C:/Users/fly/Zotero/storage/KBF5JBC7/杨凡 等 - 2016 -

KeyboardInterrupt: 

## 生成问题答案

In [29]:
# 使用RAG生成回答并保存为jsonl格式
try:
    import json
    
    # 定义输出文件
    qa_output_file = "pdf_qa_results.jsonl"
    
    if all_pdfs_qa_chain and all_questions:
        print("\n回答生成的问题并保存为JSONL格式:")
        
        # 打开文件准备写入
        with open(qa_output_file, 'w', encoding='utf-8') as f:
            # 遍历每个PDF文件的问题
            for pdf_name, questions_text in all_questions.items():
                print(f"\n处理来自 {pdf_name} 的问题:")
                
                # 尝试将问题文本分割成单独的问题
                questions = questions_text.split("\n")
                
                for question in questions:
                    # 清理问题文本
                    clean_question = re.sub(r'^\d+\.\s*', '', question).strip()
                    if clean_question and len(clean_question) > 10:  # 确保问题有足够长度
                        print(f"\n问题: {clean_question}")
                        answer_result = all_pdfs_qa_chain.invoke({"question": clean_question, "chat_history": []})
                        print(f"回答: {answer_result['answer']}")
                        
                        # 显示来源
                        print("来源文档:")
                        sources = set()
                        for doc in answer_result["source_documents"]:
                            if "source" in doc.metadata:
                                sources.add(doc.metadata["source"])
                        for source in sources:
                            print(f"- {source}")
                        
                        # 创建要保存的记录
                        qa_record = {
                            "pdf_name": pdf_name,
                            "question": clean_question,
                            "answer": answer_result['answer'],
                            "sources": list(sources)
                        }
                        
                        # 写入JSONL文件
                        f.write(json.dumps(qa_record, ensure_ascii=False) + '\n')
        
        print(f"\n问答记录已保存到: {qa_output_file}")
    else:
        print("无法生成回答：RAG智能体未创建或没有找到之前生成的问题")
        
except Exception as e:
    print(f"生成回答时出错: {str(e)}")
    import traceback
    traceback.print_exc()


回答生成的问题并保存为JSONL格式:

处理来自 黄国庆 等 - 2011 - 上海光源X射线针孔相机控制系统设计.pdf 的问题:
生成回答时出错: 'list' object has no attribute 'split'


Traceback (most recent call last):
  File "C:\Users\fly\AppData\Local\Temp\ipykernel_13748\395687644.py", line 18, in <module>
    questions = questions_text.split("\n")
AttributeError: 'list' object has no attribute 'split'


## 补全推理过程

In [30]:
# 构建Agent来补全推理过程
try:
    from langchain.chat_models import ChatOpenAI
    from langchain.schema import HumanMessage, SystemMessage
    from langchain.prompts import ChatPromptTemplate
    import json
    import os
        
    # 定义输入和输出文件
    qa_input_file = "pdf_qa_results.jsonl"  # 之前保存的问答结果文件
    reasoning_output_file = "qa_with_reasoning.jsonl"  # 包含推理过程的输出文件
    
    # 检查输入文件是否存在
    if not os.path.exists(qa_input_file):
        print(f"错误：找不到输入文件 {qa_input_file}")
    else:
        # 初始化大模型
        llm = ChatOpenAI(
            temperature=0.1,
            model_name="deepseek-ai/DeepSeek-R1"
            )
        
        # 创建提示模板
        system_message = """你是一个专业的学术助手，精通各学科领域的专业知识。你的任务是为给定的问题和答案补充详细、准确的推理过程，展示从问题到答案的完整思考路径。

请注意以下要求：
1. 推理过程必须清晰展示从问题分析到得出答案的完整思考链条
2. 基于可靠的学术原理和专业知识，不要编造信息
3. 包含具体的技术细节、公式、原理和机制解释，而非泛泛而谈
4. 使用该领域的专业术语和标准表达方式
5. 按照逻辑顺序组织内容，可以使用编号或小标题增强结构性
6. 必要时引入定量分析，包括数值范围、单位和量级
7. 解释关键参数之间的关系和影响机制
8. 明确说明每一步推理如何引导到下一步，最终如何得出答案中的结论

推理过程应该结构清晰，包含以下部分：
- 问题分析：明确问题的核心要点和需要解决的关键问题
- 技术原理：相关的专业知识和技术原理
- 工作机制：关键组件的工作机制和参数关系
- 系统设计：系统设计中的关键考量和技术路线
- 性能分析：系统性能指标和应用价值

避免使用"XX"这样的占位符，确保所有内容都是具体、明确的。不要使用模板化的结构，而是根据具体问题提供有针对性的分析。"""
        
        # 打开输入文件读取问答记录
        qa_records = []
        with open(qa_input_file, 'r', encoding='utf-8') as f:
            for line in f:
                qa_records.append(json.loads(line.strip()))
        
        print(f"读取了 {len(qa_records)} 条问答记录")
        
        # 如果DEBUG模式，只处理第一条记录
        if DEBUG:
            qa_records = qa_records[:1]
            print("DEBUG模式：只处理第一条记录")
        
        # 打开输出文件准备写入
        with open(reasoning_output_file, 'w', encoding='utf-8') as f:
            # 遍历每条问答记录
            for i, record in enumerate(qa_records):
                print(f"\n处理第 {i+1}/{len(qa_records)} 条记录...")
                
                # 获取问题和答案
                question = record["question"]
                answer = record["answer"]
                
                print(f"问题: {question[:50]}..." if len(question) > 50 else f"问题: {question}")
                print(f"答案: {answer[:100]}..." if len(answer) > 100 else f"答案: {answer}")
                
                # 构建提示并调用大模型
                human_message = f"""问题: {question}
                
答案: {answer}
                
请为上述问答补充详细的推理过程，确保清晰展示从问题到答案的完整思考路径，包含足够的专业深度和技术细节。"""
                
                messages = [
                    SystemMessage(content=system_message),
                    HumanMessage(content=human_message)
                ]

                # 打印提示信息
                print("发送到LLM的提示:")
                for msg in messages:
                    print(f"[{msg.type}]: {msg.content[:100]}..." if len(msg.content) > 100 else f"[{msg.type}]: {msg.content}")
                print("-" * 50)
                
                # 获取推理过程
                response = llm.invoke(messages)
                reasoning = response.content
                
                # 更新记录，添加推理过程
                record["reasoning"] = reasoning
                
                # 写入JSONL文件
                f.write(json.dumps(record, ensure_ascii=False) + '\n')
                
                # 打印推理过程
                print(f"推理过程:\n{reasoning}")
                print(f"已添加推理过程 ({len(reasoning)} 字符)")
        
        print(f"\n推理过程已添加并保存到: {reasoning_output_file}")
        if DEBUG:
            print("注意：由于DEBUG模式开启，只处理了第一条记录")

except Exception as e:
    print(f"补全推理过程时出错: {str(e)}")
    import traceback
    traceback.print_exc()



读取了 0 条问答记录
DEBUG模式：只处理第一条记录

推理过程已添加并保存到: qa_with_reasoning.jsonl
注意：由于DEBUG模式开启，只处理了第一条记录
